# **Sesión 1:** Calibración de Cámara 📷⚙️

## **Instalaciones**

In [1]:
!pip install numpy==1.26 opencv-python==4.8.0.76 imageio

## **Librerías**

In [30]:
from typing import List
import numpy as np
import imageio
import cv2
import copy
import glob
import os

## **Apartado A: Calibración de cámara** (derecha e izquierda)

En este apartado se realiza la calibración de dos cámaras de un sistema estereoscópico. Para ello se trabajará con las imágenes de las carpetas ``left`` y ``right``. En primer lugar se trabajará con la carpeta ``left``. Posteriormente, deberá repetir el proceso con las imágenes en la carpeta ``right``. Ambas carpetas contienen imágenes con las que se calibrarán las cámaras. En ellas aparece el patrón de calibración en diferentes posiciones y orientaciones. Estas imágenes serán los datos de entrada.

Los pasos que deberá seguir para calibrar una cámara son:

1. Defina y ejecute el método para cargar imágenes ``load_images()``.
2. Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.
3. Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``.
4. Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)`` que proporcione las coordenadas 3D de las esquinas del patrón. El sistema de referencia utilizado deberá estar anclado al propio patrón.
5. Utilice ``cv2.calibrateCamera`` para obtener los parámetros de calibración para la cámara izquierda.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``.

In [31]:
def load_images(filenames: List) -> List:
    return [imageio.imread(filename) for filename in filenames] # On the future imageio.v2.imread

In [32]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = [path for path in glob.glob( "../data/right/*jpg" )] # "../data/left/*jpg" 
imgs = load_images(imgs_path)

C:\Users\lydia\AppData\Local\Temp\ipykernel_6588\2750984346.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames] # On the future imageio.v2.imread


In [33]:
print(glob.glob( "../data/right/*jpg" ))

['../data/right\\right_000.jpg', '../data/right\\right_001.jpg', '../data/right\\right_002.jpg', '../data/right\\right_003.jpg', '../data/right\\right_004.jpg', '../data/right\\right_005.jpg', '../data/right\\right_006.jpg', '../data/right\\right_007.jpg', '../data/right\\right_008.jpg', '../data/right\\right_009.jpg', '../data/right\\right_010.jpg', '../data/right\\right_011.jpg', '../data/right\\right_012.jpg', '../data/right\\right_013.jpg', '../data/right\\right_014.jpg', '../data/right\\right_015.jpg', '../data/right\\right_016.jpg', '../data/right\\right_017.jpg', '../data/right\\right_018.jpg']


### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [34]:
# TODO Find corners with cv2.findChessboardCorners()
corners = [cv2.findChessboardCorners(img, (8,6)) for img in imgs]

In [35]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [36]:
imgs_copy = copy.deepcopy(imgs)

In [37]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
drawed_corners = [cv2.drawChessboardCorners( imgs_copy[i], (8,6), corners_refined[i], corners[i][0])  for i in range(1, len(corners)) if i!= 14] # draws on imgs_copy,  range(1, len(corners),  do not pick 14 on right

In [38]:
# TODO Show images and save when needed

def show_image(img):
    cv2.imshow("image", img)
    cv2.waitKey(0) # Screen time
    cv2.destroyAllWindows()
    
def write_image(counter_image, img):
    cv2.imwrite(f"../data/right_detected/detected_{counter_image}.jpg",img) # f"../data/left_detected/detected_{counter_image}.jpg"

os.makedirs("../data/right_detected", exist_ok = True) # "../data/left_detected"

for i in range(0, len(imgs_copy)): # 1, len(imgs_copy)
    if i != 14:
        write_image(i, imgs_copy[i]) 

### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [39]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    
    points = []
    
    for i in range(chessboard_shape[1]):
        for j in range(chessboard_shape[0]):
            points.append(np.array([i*dy, j*dx, 0]))
            
    return np.array(points, dtype = np.float32)

In [40]:
# TODO You need the points for every image, not just one
chessboard_points = [get_chessboard_points((8, 6), 30, 30) for i in range(1, len(corners_refined))] # One per each image with corners detected

### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [41]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [42]:
# TODO
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(chessboard_points, np.concatenate([corners_refined[:14], corners_refined[15:]]), imgs_gray[-1].shape[::-1], None, None) # corners_refined[1:]

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [43]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)
print(len(corners_refined))

Intrinsics:
 [[431.21056811   0.         146.62610374]
 [  0.         432.38883761 137.3134062 ]
 [  0.           0.           1.        ]]
Distortion coefficients:
 [[-0.11667294 -0.2856263   0.00665848 -0.00543096  2.06740353]]
Root mean squared reprojection error:
 0.10285746319857729
19


### **Pregunta A.1:** Repita el proceso (carga de imágenes, detección y comprobación de esquinas, etc.) para la cámara derecha.

In [44]:
# TODO Homework

### **Pregunta A.2:** Diferencias entre cv2.findChessboardCorners() y cv2.cornerSubPix()

In [45]:
# TODO Homework

### **Pregunta A.3:** Número mínimo de imágenes necesarias para calibrar.

In [46]:
# TODO Homework

## **Apartado B: Corrección de distorsión** (ojo de pez)

En este apartado se trabajará en la corrección de la distorsión debido a lentes de ojo de pez. Primero se calibrará una cámara con este tipo de lente, utilizando las imágenes de la carpeta ``fisheye``. Posteriormente se utilizarán estos parámetros de calibración para corregir la distorsión de una de las imágenes de calibración.

Los pasos que deberá seguir para calibrar una cámara con distorsión de ojo de pez son:

1. Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``.
2. Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.
3. Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero.
4. Defina los argumentos para la función de calibración.
5. Calibre con ``cv2.fisheye.calibrate()``

### **Tarea B.1:** Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``

In [47]:
# TODO Build a list containing the paths of all images from the fisheye camera and load images
fisheye_imgs_path = glob.glob('../data/fisheye/*.jpg')
fisheye_imgs = load_images(fisheye_imgs_path)

C:\Users\lydia\AppData\Local\Temp\ipykernel_6588\2750984346.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames] # On the future imageio.v2.imread


### **Tarea B.2:** Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.

In [48]:
imgs_corners = []
# Parameters for cv2.cornerSubPix()
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)

#TODO Complete the required parts of the loop
for img in fisheye_imgs:
    
    # TODO parse arguments to cv2.findChessboardCorners()
    corners = cv2.findChessboardCorners(img, (7,6))
    
    # TODO convert image to grayscale to use cv2.cornerSubPix()
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    refined_corners = cv2.cornerSubPix(gray_img, corners[1], (7,6), (-1,-1), subpix_criteria)

    # TODO append only those refined_corners with proper detections
    imgs_corners.append(refined_corners)
    
imgs_corners = np.array(imgs_corners, dtype=np.float32)

### **Tarea B.3:** Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero

In [49]:
# TODO Define the chessboard dimensions and the lenght of the squares (in [mm])
chessboard_dims = (7, 6)
length = 30
fisheye_chessboard_points = [get_chessboard_points(chessboard_dims, length, length) for _ in range(len(imgs_corners))]

In [50]:
print(fisheye_chessboard_points[0].dtype)

float32


### **Tarea B.4:** Defina los argumentos para la calibración

In [51]:
# Parameters for cv2.fisheye.calibrate()
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW # +cv2.fisheye.CALIB_CHECK_COND
intrinsics = np.zeros((3, 3))
distortion = np.zeros((4, 1))
rotations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]
traslations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]


### **Tarea B.5:** Calibración

In [52]:
rms, _, _, _, _ = \
cv2.fisheye.calibrate(np.expand_dims(np.asarray(fisheye_chessboard_points), -2), imgs_corners, gray_img.shape[::-1], intrinsics, distortion, rotations, traslations, calibration_flags, subpix_criteria)

In [53]:
# Show intrinsic matrix and distortion coefficients values
print(intrinsics)
print(distortion)

[[188.50770041   0.         504.10055382]
 [  0.         184.96088979 373.6659997 ]
 [  0.           0.           1.        ]]
[[ 0.08230382]
 [ 0.00263929]
 [ 0.03569538]
 [-0.03186732]]


### **Pregunta B.1:** Corrija la distorsión de las 2 primeras imágenes de la carpeta ``fisheye``

In [54]:
# TODO Search in the documentation to define 'dim'
dim = img.shape[:2][::-1]
map1, map2 = cv2.fisheye.initUndistortRectifyMap(intrinsics, distortion, np.eye(3), intrinsics, dim, cv2.CV_16SC2)

In [55]:
# Correct distortion using cv2.remap()
corrected_imgs = [cv2.remap(img, map1, map2, interpolation=cv2.INTER_CUBIC) for img in fisheye_imgs] # Enlarge picture: INTER_LINEAR, INTER_CUBIC. # Shrink picture: INTER_AREA

# Display corrected images
for img in corrected_imgs:
    show_image(img)

